In [5]:

import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import google_analytics_reporting_api as ga
import os



In [6]:
today_Friday=datetime.datetime.now().date()
today_weekday_num_py=today_Friday.weekday()
this_week_Monday=today_Friday-datetime.timedelta(days=today_weekday_num_py)

week_counter_str=str(int(np.ceil((today_Friday-datetime.date(2018,9,21)).days/7)))
week="_September_Week"+week_counter_str

Start_Date_2=str(this_week_Monday-datetime.timedelta(days=4))
End_Date_2=str(this_week_Monday+datetime.timedelta(days=3))

Start_Date_overall='2018-09-11'
End_Data_overall=End_Date_2

Start_Date_1='2018-09-11' # No Change
End_Date_1='2018-09-20'

today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"+week[1:]+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Google Analytics

In [7]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


In [8]:
# Saatva.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_saatva=ga_data_1.append(ga_data_2)
ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape

(2648, 9)

In [9]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_SM=ga_data_1.append(ga_data_2)
ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape

(20264, 9)

In [10]:
ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)

In [11]:
ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

In [12]:
# msn in 'social'
# mail.yahoo.com / referral in 'referral'
# Social: 'msn','instagram','pinterest','facebook
                                                
ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(((ga_data['ga:sourceMedium'].str.contains('youtube.')) | (ga_data['Medium']=="social")),"Social",
                                  np.where(((ga_data['Source'].isin(['msn','instagram','Instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc')),"Social",
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc')),"SEM",
                                                                      np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic')),"SEO",
                                                                               np.where(ga_data['Source']=="powerInbox","Email",
                                                                                        np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                 np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                          np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                   np.where(ga_data['Medium'].isin(["partner","partners"]),"partner",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )
                                                         


In [13]:
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']
GA_taxonomy.shape

(5167, 4)

In [16]:
ga_agg_session=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg_session.reset_index(inplace=True)
ga_agg_wide_session=ga_agg_session.pivot('ga:dateHour','Defined_Media','ga:sessions')
ga_agg_wide_session.reset_index(inplace=True)
ga_agg_wide_session['time_hour']=ga_agg_wide_session['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))

In [17]:
ga_agg_transaction=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:transactions'].sum().to_frame()
ga_agg_transaction.reset_index(inplace=True)
ga_agg_wide_trans=ga_agg_transaction.pivot('ga:dateHour','Defined_Media','ga:transactions')
ga_agg_wide_trans.reset_index(inplace=True)
ga_agg_wide_trans['time_hour']=ga_agg_wide_trans['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))

In [18]:
ga_agg_Revenue=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:transactionRevenue'].sum().to_frame()
ga_agg_Revenue.reset_index(inplace=True)
ga_agg_wide_Revenue=ga_agg_Revenue.pivot('ga:dateHour','Defined_Media','ga:transactionRevenue')
ga_agg_wide_Revenue.reset_index(inplace=True)
ga_agg_wide_Revenue['time_hour']=ga_agg_wide_Revenue['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))


In [ ]:
writer=pd.ExcelWriter(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})

In [20]:
# Combined both Saatva.com and Saatvamattress.com
ga_agg_wide_session.to_excel(writer,"Session_hour_media",index=False)
ga_agg_wide_trans.to_excel(writer,"Trans_hour_media",index=False)
ga_agg_wide_Revenue.to_excel(writer,"Revenue_hour_media",index=False)

ga_data.to_excel(writer,"data",index=False)
# GA_taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()

GA_taxonomy.to_csv(writer_folder+'/Saatva_S_and_SM_GA_taxonomy_'+Start_Date_1+'_to_'+End_Date_2+week+'.csv',index=False,sep="\t",encoding="utf-8")

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=s&gca_device=c&gca_adposition=1t4&gca_loc_interest_ms=&gca_loc_physical_ms=9061285&gclid=EAIaIQobChMI6ObC9u-y3QIVQrXACh379gBNEAAYBCAAEgJgMfD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51627636938&gca_matchtype=e&gca_network=s&gca_device=c&gca_adposition=1s1&gca_loc_interest_ms=&gca_loc_physical_ms=9016899&gclid=EAIaIQobChMI0P7lhfGy3QIVELnACh3rTAr1EAEYASAAEgKzQPD_BwE%20/%20partner' with link or location/anchor > 255 cha

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51627636938&gca_matchtype=e&gca_network=g&gca_device=c&gca_adposition=1t2&gca_loc_interest_ms=&gca_loc_physical_ms=9015108&gclid=EAIaIQobChMIjtmDhNC-3QIVDLjACh3hFAAREAAYAiAAEgIDqPD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://topmattressesonline.com/memory-foam?strategy=home&utm_source=google&utm_term=best%20memory%20foam%20mattress&externalcreative=287527496156&p=&utm_content=1o3&utm_ad=60785436231&utm_medium=kwd-908244988&externalcampaign=1506069980&matchtype=e&gclid=EAIaIQobChMI96jjh4q_3QIVz0wNCh3J_QUOEAMYAyAAEgKuO_D_BwE%20/%20partner' with link or location/anchor > 255 characte

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=CjwKCAjwio3dBRAqEiwAHWsNVbpz32IlVBdx9UnTkqum2ZLERxx7Up4WxBnz_u-if99RRdtfroCvshoC2N0QAvD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=CjwKCAjwio3dBRAqEiwAHWsNVVdjq6S13Q40weSrTHRSbc9qTbyieqh03h1Y2kCe-H6sRVVdJ6VcDxoC4JAQAvD_BwE%20/%20partner' with link or locatio

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.mattressadvisor.com/best-mattress/?k_clickid=_k_CjwKCAjw6-_eBRBXEiwA-5zHabuz1KyXF3TtAx9mRYVXW49k471kRn87MI_7dslxbrtz7M9vYvBAcxoCtDwQAvD_BwE_k_&adgroup=Best_Mattress_2018&gclid=CjwKCAjw6-_eBRBXEiwA-5zHabuz1KyXF3TtAx9mRYVXW49k471kRn87MI_7dslxbrtz7M9vYvBAcxoCtDwQAvD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.mattressadvisor.com/best-mattress/?k_clickid=_k_CjwKCAjw6-_eBRBXEiwA-5zHacR0pGDIJptQfpHeNPwpzcXZ1e8xOdn4mN-151vVI6EsjlKLgx5NlxoCTc8QAvD_BwE_k_&adgroup=Best_Mattress_2018&gclid=CjwKCAjw6-_eBRBXEiwA-5zHacR0pGDIJptQfpHeNPwpzcXZ1e8xOdn4mN-151vVI6EsjlKLgx5NlxoCTc8QAvD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?pd=true&keyword=mattress%20in%20a%20box%20reviews&gca_campaignid=1016369000&gca_adgroupid=51724296869&gca_matchtype=e&gca_network=g&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9030943&&pd=true&keyword=mattress%20in%20a%20box%20reviews&gca_campaignid=1016369000&gca_adgroupid=51724296869&gca_matchtype=e&gca_network=g&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9030943&gclid=EAIaIQobChMI4vqd-fq93gIVj8hkCh0alwjTEAAYASAAEgJnifD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.mattressadvisor.com/best-mattress-2/?k_clickid=_k_Cj0KCQiA8f_eBRDcARIsAEKwRGcjuXcHzmBwqFw2xbCEfTkSXXA9g6gJUvovNy8viy14jWZbmHppF

...

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-9586635656353368&output=html&h=60&slotname=7331567408&adk=3448569538&adf=546890593&w=722&fwr_io=true&fwrn=4&fwrnh=100&lmt=1536743910&rafmt=1&guci=2.2.0.0.2.2.0&format=722x60&url=https://www.925-1000.com/dlc_birmingham.html&flash=0&fwr=0&fwrattr=true&rh=0&rw=722&resp_fmts=3&wgl=1&dt=1537015859783&bpp=17&bdt=78&fdt=20&idt=30&shv=r20180910&cbv=r20180604&saldr=aa&abxe=1&correlator=1737465235387&frm=20&pv=2&ga_vid=1173885443.1537015799&ga_sid=1537015799&ga_hid=911710249&ga_fc=1&iag=0&icsg=680&dssz=10&mdo=0&mso=0&u_tz=-240&u_his=1&u_java=0&u_h=1080&u_w=1920&u_ah=1040&u_aw=1920&u_cd=24&u_nplug=3&u_nmime=4&adx=16&ady=740&biw=753&bih=700&scr_x=0&scr_y=0&eid=21060853,828064254&oid=3&ref=https://www.925-1000.com/dlBirmingham5.html&rx=0&eae=0&fc=528&brdim=0,0,0,0,1920,0,786,767,770,700&vis=1&rsz=||eEbr|&abl=CS&ppjl=f&pfx=0&fu=144&bc=7&osw_key=2

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-1408961244894751&output=html&h=280&slotname=1176089194&adk=2942176734&adf=3839756744&w=350&fwrn=4&fwrnh=100&lmt=1537285319&rafmt=1&guci=2.2.0.0.2.2.0&format=350x280&url=https://sweepstakesfanatics.com/yamaha-pick-passion-sweepstakes/&flash=0&fwr=0&rh=0&rw=350&resp_fmts=3&wgl=1&dt=1537285319354&bpp=49&bdt=2099&fdt=56&idt=74&shv=r20180912&cbv=r20180604&saldr=aa&abxe=1&prev_fmts=0x0,571x60&nras=1&correlator=8755857863745&frm=20&pv=1&ga_vid=683202767.1532200723&ga_sid=1537285319&ga_hid=1290665880&ga_fc=0&iag=0&icsg=10737562282&dssz=32&mdo=0&mso=0&u_tz=-300&u_his=7&u_java=0&u_h=962&u_w=601&u_ah=962&u_aw=601&u_cd=24&u_nplug=0&u_nmime=0&adx=114&ady=617&biw=601&bih=874&scr_x=0&scr_y=0&eid=21060853&oid=3&ref=https://sweepstakesfanatics.com/expires-tomorrow/&rx=0&eae=0&fc=1812&brdim=0,0,0,0,601,0,601,874,601,874&vis=1&rsz=||eE|&abl=CS&ppjl=f&

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=g&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9004606&&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=g&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9004606&gclid=CjwKCAjw6-_eBRBXEiwA-5zHaSIEyChqB8YOVhauAZ-wRpPV-axhfu9uEtSXQR1fK-sWwwK47Bq9pxoCjFQQAvD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.mattressadvisor.com/best-mattress/?k_clickid=_k_CjwKCAjw6-_eBRBXEiwA-5zHaXfIM_iJmEZkayhNUVpakdxMtTJ0w0rOfie2fhiVPtB4AMnMbOzBdhoCggMQAvD_BwE_k

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?pd=true&keyword=mattress%20in%20a%20box%20reviews&gca_campaignid=1016369000&gca_adgroupid=51724296869&gca_matchtype=e&gca_network=g&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9030943&&pd=true&keyword=mattress%20in%20a%20box%20reviews&gca_campaignid=1016369000&gca_adgroupid=51724296869&gca_matchtype=e&gca_network=g&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9030943&gclid=EAIaIQobChMI4vqd-fq93gIVj8hkCh0alwjTEAAYASAAEgJnifD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.mattressadvisor.com/best-mattress-2/?k_clickid=_k_Cj0KCQiA8f_eBRDcARIsAEKwRGcjuXcHzmBwqFw2xbCEfTkSXXA9g6gJUvovNy8viy14jWZbmHppFpIaAvQEEALw_wc

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?gclsrc=aw.ds&&pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20-%20Superlatives%20(Exact)&pt_adgroup=best%20mattress%202017&pt_keyword=best%20mattress%202018&gclid=Cj0KCQiA2o_fBRC8ARIsAIOyQ-mZh88Hyq9Xg8W4bVWrJROinQlg_sJd7QIwkc_hX3wdMH_4t3mP_X8aAiR3EALw_wcB' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.mattressadvisor.com/mattress-reviews/saatva-mattress-review/?k_clickid=_k_Cj0KCQiA2o_fBRC8ARIsAIOyQ-mKhvwL97mvFxmX4NtcjJRRaWtKXAmkIWeRfGPK5c1C3M8nOlk9JUAaAtl-EALw_wcB_k_&adgroup=Mattress_Reviews&gclid=Cj0KCQiA2o_fBRC8ARIsAIOyQ-mKhvwL97mvFxmX4NtcjJRRaWtKXAmkIWeRfGPK5c1C3M8nOlk9JUAaAtl-EALw_wcB' with link or location/anchor > 255 cha

In [22]:
media_col=ga_agg_wide_session.columns.tolist()[1:(len(ga_agg_wide_session.columns)-1)]

In [23]:
def write_csv_output(df,Matrics):
    ga_output_csv=df.copy()
    ga_output_csv['Date']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[0:8]))
    ga_output_csv['Hour']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[8:10]))
    del ga_output_csv['ga:dateHour']
    ga_output_csv=ga_output_csv.sort_values(['Date','Hour'],ascending=True)
    ga_output_csv=ga_output_csv[['Date','Hour']+media_col]
    ga_output_csv=ga_output_csv.fillna(0)
    ga_output_csv.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+"_"+Matrics+'.csv',index=False)

In [24]:
write_csv_output(ga_agg_wide_session,"Session")
write_csv_output(ga_agg_wide_trans,"Trans")
write_csv_output(ga_agg_wide_Revenue,"Revenue")


In [25]:
ga_agg_session_by_View=ga_data.groupby(['View','View_ID','ga:dateHour','Defined_Media'])['ga:sessions'].sum().unstack('Defined_Media')
ga_agg_session_by_View.reset_index(inplace=True)
ga_agg_session_by_View=ga_agg_session_by_View.fillna(0)
ga_agg_session_by_View.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_by_View'+Start_Date_1+'_to_'+End_Date_2+week+"_Sessions"+'.csv',index=False)


# Google AdWords

In [30]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [31]:
client_Saatva = get_client(config_path='/mnt/drv5/jian/Saatva/weekly_celery/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Click['Channel_Network']=Saatva_Click['Advertising Channel']+"|"+Saatva_Click['Network']+"|"+Saatva_Click['Network (with search partners)']

In [32]:
client_Saatva_Branding = get_client(config_path='/mnt/drv5/jian/Saatva/weekly_celery/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Brand_Click['Channel_Network']=Saatva_Brand_Click['Advertising Channel']+"|"+Saatva_Brand_Click['Network']+"|"+Saatva_Brand_Click['Network (with search partners)']
Saatva_Brand_Click['Channel_Network'].unique()

array(['Display|Display Network|Display Network',
       'Video|YouTube Search|YouTube Search'], dtype=object)

In [33]:
client_Saatva_Competitors = get_client(config_path='/mnt/drv5/jian/Saatva/weekly_celery/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Competitor_Click['Channel_Network']=Saatva_Competitor_Click['Advertising Channel']+"|"+Saatva_Competitor_Click['Network']+"|"+Saatva_Competitor_Click['Network (with search partners)']



In [34]:
client_TheSaatvaCompany = get_client(config_path='/mnt/drv5/jian/Saatva/weekly_celery/googleads_saatva_JL_TheSaatvaCompany.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompany = download_report_as_stream(client_TheSaatvaCompany, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompany_Click = pd.read_csv(response_TheSaatvaCompany)
TheSaatvaCompany_Click['Cost']=TheSaatvaCompany_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompany_Click['Channel_Network']=TheSaatvaCompany_Click['Advertising Channel']+"|"+TheSaatvaCompany_Click['Network']+"|"+TheSaatvaCompany_Click['Network (with search partners)']



In [35]:
client_TheSaatvaCompanyBranding = get_client(config_path='/mnt/drv5/jian/Saatva/weekly_celery/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompanyBranding = download_report_as_stream(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompanyBranding_Click = pd.read_csv(response_TheSaatvaCompanyBranding)
TheSaatvaCompanyBranding_Click['Cost']=TheSaatvaCompanyBranding_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompanyBranding_Click['Channel_Network']=TheSaatvaCompanyBranding_Click['Advertising Channel']+"|"+TheSaatvaCompanyBranding_Click['Network']+"|"+TheSaatvaCompanyBranding_Click['Network (with search partners)']



In [36]:
Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']
# Keep only campaign labeled as "[SM]" in Saatva Competitor account
# Broad and Exact ADL Non-Brand Search Partners are not included

In [37]:
set(Saatva_Click['Channel_Network'].unique().tolist()+Saatva_Brand_Click['Channel_Network'].unique().tolist()+Saatva_Competitor_Click['Channel_Network'].unique().tolist()+\
    TheSaatvaCompany_Click['Channel_Network'].unique().tolist()+TheSaatvaCompanyBranding_Click['Channel_Network'].unique().tolist())

{'Display|Display Network|Display Network',
 'Search|Search Network|Google search',
 'Search|Search Network|Search partners',
 'Shopping|Search Network|Google search',
 'Shopping|Search Network|Search partners',
 'Video|Display Network|Display Network',
 'Video|YouTube Search|YouTube Search',
 'Video|YouTube Videos|YouTube Videos'}

In [38]:
df_all_adwords=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
df_all_adwords=df_all_adwords.append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)

In [39]:
df_s=df_all_adwords.groupby(['Channel_Network'])[['Impressions','Clicks','Cost']].sum().reset_index()
Video_summary=df_s[df_s['Channel_Network'].apply(lambda x: "Video" in x)]

,Channel_Network,Impressions,Clicks,Cost
5,Video|Display Network|Display Network,55817,126,114.438362
6,Video|YouTube Search|YouTube Search,835903,57,2658.320000
7,Video|YouTube Videos|YouTube Videos,23715172,130499,320905.694238


In [52]:
'''
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompany_Click=TheSaatvaCompany_Click[~((TheSaatvaCompany_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompanyBranding_Click=TheSaatvaCompanyBranding_Click[~((TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Zenhaven")))]
'''
# No need to exclude the compaigns because of the small pctg

Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type
TheSaatvaCompany_Click['Campaign_Type']=TheSaatvaCompany_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
TheSaatvaCompanyBranding_Click['Campaign_Type']=TheSaatvaCompanyBranding_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))


In [53]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
len(Saatva_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [54]:
# Split TheSaatvaCompany

TheSaatvaCompany_Click_Search=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']=='Search']
TheSaatvaCompany_Click_Others=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']!='Search']

TheSaatvaCompany_Click_Search['Campaign_Type']=np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Branded")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains(" Brand ")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Co-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Non-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
TheSaatvaCompany_Click_Search_1=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']!="NA"]
TheSaatvaCompany_Click_Search_2=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']=="NA"]
len(TheSaatvaCompany_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [55]:
# All campaing is able to differentiate from the name about brand and non-brand, so no need to go to keywords level
'''
Saatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)
Saatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)
Saatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)

Saatva_keywords=Saatva_keywords[~((Saatva_keywords['Campaign'].str.contains("Loom")) | (Saatva_keywords['Campaign'].str.contains("Leaf")) | (Saatva_keywords['Campaign'].str.contains("Zenhaven")))]
Saatva_keywords['Campaign_Type']=np.where(Saatva_keywords['Search keyword'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].astype(int)
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].astype(int)
Saatva_keywords=Saatva_keywords.groupby(['Campaign','Campaign_Type'])['Impressions','Clicks'].sum()
Saatva_keywords.reset_index(inplace=True)
Saatva_keywords=Saatva_keywords.sort_values(['Campaign','Impressions'])
Saatva_keywords=Saatva_keywords.drop_duplicates('Campaign')
Saatva_Campaign_Type=Saatva_keywords[['Campaign','Campaign_Type']]

del Saatva_Click_Search_2['Campaign_Type']
Saatva_Click_Search_2=pd.merge(Saatva_Click_Search_2,Saatva_Campaign_Type,on="Campaign",how="left")

Saatva_Search_Click=Saatva_Click_Search_1.append(Saatva_Click_Search_2)
Saatva_Search_Click['Campaign_Type']=Saatva_Search_Click['Campaign_Type'].fillna("All_0_Values_and_No_Kws")
'''

'\nSaatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)\nSaatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)\nSaatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)\n\nSaatva_keywords=Saatva_keywords[~((Saatva_keywords[\'Campaign\'].str.contains("Loom")) | (Saatva_keywords[\'Campaign\'].str.contains("Leaf")) | (Saatva_keywords[\'Campaign\'].str.contains("Zenhaven")))]\nSaatva_keywords[\'Campaign_Type\']=np.where(Saatva_keywords[\'Search keyword\'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].apply(lambda x: int(x.replace(",","")))\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].astype(int)\nSaatva_keywords[\'Clicks\']=Saatva_keywords[\'Clicks\'].apply(

In [56]:
Saatva_Click_Search_2.head(2)
print_str_0="Make sure it's 0: "+str(len(Saatva_Click_Search_2))
print print_str_0

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network,Campaign_Type


In [57]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)
TheSaatvaCompany_Click=TheSaatvaCompany_Click_Search.append(TheSaatvaCompany_Click_Others)


In [58]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.xlsx",engine='xlsxwriter')

In [59]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg['Hour of day']=Saatva_Agg['Hour of day'].astype(int)
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [60]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day","Hour of day"])['Impressions','Clicks', 'Cost'].apply(sum)
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by hour day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]
Video_summary.to_excel(writer,"Video_in_the_Note",index=False)
Saatva_Agg_No_NA.to_excel(writer,"clean data by hour day",index=False)

In [61]:
writer.save()

In [62]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Hour of day','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Hour of day":"Hour","Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date","Hour"],ascending=[True,True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.csv",index=False)

In [63]:
Saatva_Agg_Category.to_csv(writer_folder+"/Saatva_AdWords_Taxonomy_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.csv",index=False)
